# The importance of spell checkers

<img src="https://pics.me.me/said-lunch-not-launch-imgflip-con-why-spellcheck-is-critical-30239087.png">

In 1975 Steve Johnson of Bell Labs wrote the first version of `spell` in an afternoon, which would later become the standard Unix spelling checker for the English language.

``` bash
prepare filename |                 # remove formatting commands
    translit A-Z a-z |             # map upper to lower case
        translit ^a-z @n |         # remove punctuation
            sort |                 # put words in alphabetical order
                unique |           # remove duplicate words
                    common -2 dict # report words not in dictionary
```
(from programming pearls Jon Bentley ...) `# TODO cite correctly`

In [97]:
from typing import Set, Tuple, Iterable, Callable
import re
import timeit

from profilehooks import timecall, profile

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The standard english dictionary file on unix systems `/usr/share/dict/words` is quite big, consisting of 99171 different words taking up 920K of space. This is not a major issue with modern computers that have memory in the gigabytes. The big file size can be attributed to the very basic structure of the dictionary, it is simply a (comprehensive) list of english words which doesn't utilize affix- or prefix analysis.  
This means that the list contains many sections like this:  
> Abyssinia  
> Abyssinia's  
> Abyssinian  
> Abyssinian's  

In [3]:
@timecall
def spell(filename: str, words = '/usr/share/dict/words') -> Set[str]:
    with open(filename) as f, open(words) as w:
        w = set(w.read().lower().split('\n'))
        return set(re.sub(r"[^a-z]", "\n", f.read().lower()).split('\n')) - w

`spell` like spelling checker in just 4 lines of python (excl. imports) by utilizing the standard libraries.   
Obvious shortcomings like converting `isn't` to `isn` and `t` because punctuation is stripped.

In [394]:
len(spell('big.txt'))


  spell (<ipython-input-3-a74301b96c53>:1):
    0.599 seconds



5065

The speed is actually quite good, compared to the already optimized version of `spell` by Doug McIlroy (written in 1978). That version managed to check a 5,000 word document in **under 30 seconds** (on a VAX-11/750 with 3.125 MHz). The dramatic decrease in spell checking time (30 seconds for 5,000 words compared to ~0.14 seconds for nearly 100,000 words) can be largely attributed to tremendous increase in computing power.

### But can we do even better by choosing more *efficient datastructures*?  

<div style="float:left">
    <img src="Trie.svg">
    [Trie.svg](https://de.wikipedia.org/wiki/Trie#/media/File:Trie.svg) created by [nd](https://de.wikipedia.org/wiki/Benutzer:Nd) ([CC BY-SA 3.0](https://creativecommons.org/licenses/by-sa/3.0/))
</div>
We can use a **Trie** or "prefix-tree", which is an ordered tree data structure, to efficiently store all the words from our dictionary.  
The nodes themselves don't carry information about the key they're storing, instead the key is solely encoded in the position of the node and the information associated with the edges inside it.

Simple Node object that solely stores information about neighbouring nodes and which letter the the connecting edges represent.

In [5]:
class IllegalTrieEdge(Exception):
    pass

class Node:
    def __init__(self):
        self.edges = {}
        self.end_of_word = False  # note that not only leaves can be valid words, but also intermediate nodes
        
    def add_edge(self, ch: str):
        if len(ch) != 1:
            raise IllegalTrieEdge('Edges can only consist of one character')
        if ch not in self.edges:
            self.edges[ch] = Node()
        return self.edges[ch]
    
    def get_edge(self, ch: str):
        return self.edges.get(ch)
    
    def __contains__(self, edge: str):
        return edge in self.edges

Very basic implementation of a prefix tree. 

In [6]:
class Trie:
    def __init__(self, words=[]):
        self.root = Node()
        self._build_trie(words)
        
    def _find_last_prefix_node(self, word: str) -> Tuple[str, Node]:
        v = self.root
        prefix = ''
        while len(word) > 0 and word[0] in v:
            prefix += word[0]
            v = v.get_edge(word[0])
            word = word[1:]
        return prefix, v
    
    def _add_word(self, word: str) -> bool:
        prefix, node = self._find_last_prefix_node(word)
        if prefix == word:
            node.end_of_word = True
            return False
        word = word[len(prefix):]
        while len(word) > 0:
            node = node.add_edge(word[0])
            word = word[1:]
        node.end_of_word = True
        return True
    
    def _build_trie(self, words: Set[str]):
        for word in words:
            self._add_word(word)
            
    def __contains__(self, word: str) -> bool:
        prefix, node = self._find_last_prefix_node(word)
        return prefix == word and node.end_of_word
    

#### So lets test it, shall we?  
As the dictionary to store inside the trie we'll again use the standard english words file on unix systems, splitted at newlines.

In [23]:
with open('/usr/share/dict/words') as w:
    word_file_str = w.read()  
    words = [word for word in word_file_str.split('\n') if len(word) > 0]

So lets quickly confirm that the implementation is correct.

In [187]:
trie = Trie(words)
[word for word in words if word not in trie]
'adasedadwa' not in trie
'Hallo' not in trie

[]

True

True

### Memory decrease by using a prefix tree (trie)

In [27]:
from pympler import asizeof
size_word_file = asizeof.asizeof(word_file_str)
size_words = asizeof.asizeof(words)
size_trie = asizeof.asizeof(trie)
f"Size read file: {size_word_file} B; size word list: {size_words} B; size of trie: {size_trie} B"
f"Increase in size by factor {size_trie/size_word_file}"

'Size read file: 938664 B; size word list: 6884128 B; size of trie: 93362984 B'

'Increase in size by factor 99.46368881729778'

<div style="float: left; margin-right: 20px">
    <img src="https://media.giphy.com/media/3o7btPCcdNniyf0ArS/giphy-downsized.gif">
</div>
### An increase by a factor of ~ 100 ?!

Python is a highly dynamic language where **everything** is an object, even simple things like an integer or character. This adds an immense overhead, which results in this dramatic increase in size.  
So saving space by using a prefix tree implemented in native Python is not possible, due to the limitations of the language.  


#### But maybe it is at least fast?  

We'll adapt the `spell` function from earlier to calculate $A \backslash B$ where $A = \{w | w$ in file$\}$ and 
$B = \{w | w$ in dictionary$\}$ and $B$ is a Python object implementing the `__contains__` method.

In [75]:
@timecall
def spell_G(filename: str, word_dict) -> Set[str]:
    with open(filename) as f:
        return {
            word for word in
            # read the file to check -> convert to lower -> split at newlines 
            # -> remove punctuation -> remove double occurences (conv. to set)
            set(re.sub(r"[^a-z']", "\n", f.read().lower()).split('\n'))
            if word not in word_dict
        }

In [401]:
len(spell_G('big.txt', trie))


  spell_G (<ipython-input-75-517a3c4f3e63>:1):
    0.776 seconds



7435

In [404]:
import marisa_trie
trie_efficient = marisa_trie.Trie(words)
len(spell_G('big.txt', trie_efficient))


  spell_G (<ipython-input-75-517a3c4f3e63>:1):
    0.558 seconds



7435

In [405]:
import dawg
base_dawg = dawg.DAWG(words)
len(spell_G('big.txt', base_dawg))


  spell_G (<ipython-input-75-517a3c4f3e63>:1):
    0.551 seconds



7435

### Levenshtein distance (edit distance)



\begin{equation}
\qquad\operatorname{lev}_{a,b}(i,j) = \begin{cases}
  \max(i,j) & \text{ if} \min(i,j)=0, \\
  \min \begin{cases}
          \operatorname{lev}_{a,b}(i-1,j) + 1 \\
          \operatorname{lev}_{a,b}(i,j-1) + 1 \\
          \operatorname{lev}_{a,b}(i-1,j-1) + 1_{(a_i \neq b_j)}
       \end{cases} & \text{ otherwise.}
\end{cases}
\end{equation}  

where $1_{(a_i \neq b_j)}$ is an indicator function equal to 0 when $a_i = b_j$ and equal to 1 otherwise.

In [93]:
def levenshtein_distance(s: str, t: str):
    if not(s and t):    # use falsy value of '' 
        return len(s) or len(t)
    
    cost = 0 if s[-1] == t[-1] else 1
    
    return min(levenshtein_distance(s[:-1], t) +1,
               levenshtein_distance(s, t[:-1]) +1,
               levenshtein_distance(s[:-1], t[:-1]) + cost
              )

In [101]:
@timecall
def close_strings(s: str, words: Iterable[str],
                  distace_func: Callable[[str, str], int],
                  max_distance: int):
    return [word for word in words if 0 < distace_func(s, word) <= max_distance]

In [406]:
close_strings('hi', words, levenshtein_distance, 1)[:10]


  close_strings (<ipython-input-101-fef93a3e73b0>:1):
    11.713 seconds



['Bi', 'Chi', 'Ci', 'Di', 'Li', 'Ni', 'Si', 'Ti', 'chi', 'h']

In [178]:
import numpy as np
def levenshtein_dp(s: str, t: str):
    n = len(s) + 1
    m = len(t) + 1
    d = np.zeros((n, m))
    d[:, 0] = range(n)
    d[0, :] = range(m)
    
    for j in range(1, m):
        for i in range(1, n):
            cost = 0 if s[i-1] == t[j-1] else 1
            d[i, j] = min(d[i-1, j] + 1,
                          d[i, j-1] + 1,
                          d[i-1, j-1] + cost
                         )
    return d[n-1,m-1]

### Damerau-Levenshtein distance
<br>
\begin{equation}
d_{a,b}(i,j) = \begin{cases}
  \max(i,j) & \text{ if} \min(i,j)=0, \\
\min \begin{cases}
          d_{a,b}(i-1,j) + 1 \\
          d_{a,b}(i,j-1) + 1 \\
          d_{a,b}(i-1,j-1) + 1_{(a_i \neq b_j)} \\
          d_{a,b}(i-2,j-2) + 1
       \end{cases} & \text{ if } i,j > 1 \text{ and } a_i = b_{j-1} \text{ and } a_{i-1} = b_j \\
  \min \begin{cases}
          d_{a,b}(i-1,j) + 1 \\
          d_{a,b}(i,j-1) + 1 \\
          d_{a,b}(i-1,j-1) + 1_{(a_i \neq b_j)}
       \end{cases} & \text{ otherwise.}
\end{cases}
\end{equation}  
<br>
Extend `levenshtein_dp()` algorithm to include transpositions.

In [408]:
def damerau_levenshtein(s: str, t: str):
    n = len(s) + 1
    m = len(t) + 1
    d = np.zeros((n, m))
    d[:, 0] = range(n)
    d[0, :] = range(m)
    
    for j in range(1, m):
        for i in range(1, n):
            cost = 0 if s[i-1] == t[j-1] else 1
            d[i, j] = min(d[i-1, j] + 1,
                          d[i, j-1] + 1,
                          d[i-1, j-1] + cost
                         )
            if i > 1 and j > 1 and s[i-1] == t[j-2] and s[i-2] == t[j-1]:
                d[i,j] = min(d[i,j], d[i-2, j-2] + cost)
    return d[n-1,m-1]

In [409]:
out1 = close_strings('ih', words, damerau_levenshtein, 1)


  close_strings (<ipython-input-101-fef93a3e73b0>:1):
    4.639 seconds



In [410]:
out2 = close_strings('ih', words, levenshtein_dp, 1)


  close_strings (<ipython-input-101-fef93a3e73b0>:1):
    4.581 seconds



In [412]:
set(out1) - set(out2)

{'hi'}

In [413]:
with open('count_1w.txt') as cnt, open('words.txt') as w:
    cnt = [cn for cn in cnt.read().split('\n') if cn]
    w = [w for w in w.read().split('\n') if w]

In [414]:
len(spell('big.txt', 'words.txt'))


  spell (<ipython-input-3-a74301b96c53>:1):
    0.740 seconds



1861

In [374]:
w_dict = {word: 1 for word in w}

In [386]:
len(spell_G('big.txt', w_dict))


  spell_G (<ipython-input-75-517a3c4f3e63>:1):
    0.582 seconds



7189

In [ ]:
!less count_1w.txt

the     23135851162
of      13151942776
and     12997637966
to      12136980858
a       9081174698
in      8469404971
for     5933321709
is      4705743816
on      3750423199
that    3400031103
by      3350048871
this    3228469771
with    3183110675
i       3086225277
you     2996181025
it      2813163874
not     2633487141
or      2590739907
be      2398724162
are     2393614870
from    2275595356
at      2272272772
as      2247431740
count_1w.txt